# Algorithmic Bias - Core Code
Some code to get started on the Algorithmic Bias assignment. 

In [ ]:
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
bcDB = datasets.load_breast_cancer()
bcDF = pd.DataFrame(bcDB.data, columns= list(bcDB['feature_names']))
bcDF['target'] = pd.Series(bcDB.target)
bcDF = bcDF.sort_values(by = ['target'])
bcDF = bcDF.reset_index(drop=True)
bcDF.head(5)

In [ ]:
vc = bcDF['target'].value_counts()
for i,j in enumerate(bcDB.target_names):
    print (vc[i],j)

In [ ]:
y = bcDF.pop('target').values
X = bcDF.values
X.shape, y.shape

### Code to plot matrix

In [ ]:
import matplotlib.pyplot as plt
from sklearn.utils.multiclass import unique_labels
%matplotlib inline

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

## $k$-NN
#### Malignant is the minority class at ~40%.  
#### $k$-NN classifier picks up this under-representation and accentuates it, predicting just 36% malignant. 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

kNN = KNeighborsClassifier(n_neighbors=3)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=2)
y_pred_knn = kNN.fit(X_train, y_train).predict(X_test)

print("Malignant in test set : %0.2f" % (1- (y_test.sum()/len(y_test))))
print("Predicted malignant : %0.2f" % (1- (y_pred_knn.sum()/len(y_pred_knn))))

#bias?

acc = accuracy_score(y_test, y_pred_knn)
print("Accuracy hold-out: {0:.2f}".format(acc)) 
confusion = confusion_matrix(y_test, y_pred_knn)
print("Confusion matrix:\n{}".format(confusion))


# Plot normalized confusion matrix
plot_confusion_matrix(y_test, y_pred_knn, classes=['Not Helpful','Helpful'], normalize=True,
                      title='Normalized confusion matrix')


## Decision Tree

#### Decision Tree classifier picks up this under-representation of malignant but does a pretty good job of predicting 39% malignant. 

In [ ]:
from graphviz import Source
from sklearn.tree import DecisionTreeClassifier, export_graphviz

ap_features = bcDF.columns

tree = DecisionTreeClassifier(criterion='entropy')
bc_tree = tree.fit(X_train, y_train)
y_pred_tree = bc_tree.predict(X_test)

print("Malignant in test set : %0.2f" % (1- (y_test.sum()/len(y_test))))
print("Predicted malignant : %0.2f" % (1- (y_pred_tree.sum()/len(y_pred_tree))))

acc = accuracy_score(y_test, y_pred_tree)
print("Accuracy hold-out: {0:.2f}".format(acc)) 
confusion = confusion_matrix(y_test, y_pred_tree)
print("Confusion matrix:\n{}".format(confusion))


# Plot normalized confusion matrix
plot_confusion_matrix(y_test, y_pred_tree, classes=['Not Helpful','Helpful'], normalize=True,
                      title='Normalized confusion matrix')

## Logistic Regression

#### Logistic Regression classifier picks up this under-representation of malignant but predicts just 36% malignant. 

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

y_pred_lr = log_reg.predict(X_test)

print("Malignant in test set : %0.2f" % (1- (y_test.sum()/len(y_test))))
print("Predicted malignant : %0.2f" % (1- (y_pred_lr.sum()/len(y_pred_lr))))

acc = accuracy_score(y_test, y_pred_lr)
print("Accuracy hold-out: {0:.2f}".format(acc)) 
confusion = confusion_matrix(y_test, y_pred_lr)
print("Confusion matrix:\n{}".format(confusion))

# Plot normalized confusion matrix
plot_confusion_matrix(y_test, y_pred_lr, classes=['Not Helpful','Helpful'], normalize=True,
                      title='Normalized confusion matrix')

## Naives Bayes

#### Naives Bayes classifier picks up this under-representation of malignant and predicts just 37% malignant.

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

y_pred_gnb = gnb.predict(X_test)

print("Malignant in test set : %0.2f" % (1- (y_test.sum()/len(y_test))))
print("Predicted malignant : %0.2f" % (1- (y_pred_gnb.sum()/len(y_pred_gnb))))

acc = accuracy_score(y_test, y_pred_gnb)
print("Accuracy hold-out: {0:.2f}".format(acc)) 
confusion = confusion_matrix(y_test, y_pred_gnb)
print("Confusion matrix:\n{}".format(confusion))

# Plot normalized confusion matrix
plot_confusion_matrix(y_test, y_pred_gnb, classes=['Not Helpful','Helpful'], normalize=True,
                      title='Normalized confusion matrix')

## Bias Check using Cross Validation

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer

def tp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 1]
def tn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 0]
def fp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 1]
def fn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 0]
scoring = {'tp' : make_scorer(tp), 'tn' : make_scorer(tn),
           'fp' : make_scorer(fp), 'fn' : make_scorer(fn)}

models = [kNN, bc_tree, log_reg, gnb]

folds = 4
v = 0 #  use 1 or 0

for m in models:
    cv_results = cross_validate(m, X, y, cv= folds,scoring=scoring, return_train_score=False, 
                                    verbose = v, n_jobs = -1)
    fp_rate = cv_results['test_fp'].sum()/(cv_results['test_fp'].sum()+cv_results['test_tn'].sum())
    tp_rate = cv_results['test_tp'].sum()/(cv_results['test_tp'].sum()+cv_results['test_fn'].sum())
  
    print("{} x CV {:22} FP: {:.2f}  TP: {:.2f}".format(folds, type(m).__name__, fp_rate, tp_rate)) 

#### Cross Validation testing results are as follows:
##### KNN : 0.96,  Decision Tree : 0.94,  Logistic Regression : 0.97,  GaussianNB : 0.97
#### False positive denotes the bias of the model towards Benign from the above results we can see that GaussianNB and Logistic Regression have the highest True Positives at 97%. However, since Logistic Regression predicts the least False Positives, it is clearly a better model than the rest.



# 2. Propose a strategy to rectify this bias. Evaluate the effect of this strategy in terms of classification bias and overall accuracy. You may choose to work with hold-out testing

#### The bias can be varified in various methods. Here, I have used a couple of methods which are upsampling and downlsampling. 

In [ ]:
import pandas as pd
from sklearn.utils import resample
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score


bcDB = datasets.load_breast_cancer()
bcDF = pd.DataFrame(bcDB.data, columns= list(bcDB['feature_names']))
bcDF['target'] = pd.Series(bcDB.target)
bcDF = bcDF.sort_values(by = ['target'])
bcDF = bcDF.reset_index(drop=True)
y = bcDF.target
X = bcDF.drop('target', axis=1)

kNN = KNeighborsClassifier(n_neighbors=3)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# concatenate our training data back together
X = pd.concat([X_train, y_train], axis=1)

# separate minority and majority classes
mel = X[X.target==1]
beg = X[X.target==0]


print('**** Upsampling ****')

# upsample minority
beg_upsampled = resample(beg,
                          replace=True, # sample with replacement
                          n_samples=len(mel), # match number in majority class
                          random_state=27) # reproducible results
# combine majority and upsampled minority
upsampled = pd.concat([mel, beg_upsampled])

y_train = upsampled.target
X_train = upsampled.drop('target', axis=1)

y_pred_knn = kNN.fit(X_train, y_train).predict(X_test)

print("Malignant in test set : %0.2f" % (1- (y_test.sum()/len(y_test))))
print("Predicted malignant : %0.2f" % (1- (y_pred_knn.sum()/len(y_pred_knn))))

acc = accuracy_score(y_test, y_pred_knn)
print("Upsampling Accuracy hold-out: {:.2f}".format(acc)) 
confusion = confusion_matrix(y_test, y_pred_knn)
print("Upsampling Confusion matrix:\n{}".format(confusion))



print('**** Downsampling ****')

#downsample majority
mel_downsampled = resample(mel,
                          replace=True, # sample with replacement
                          n_samples=len(beg), # match number in majority class
                          random_state=27) # reproducible results
# combine majority and upsampled minority
downsampled = pd.concat([beg, mel_downsampled])

y_train = downsampled.target
X_train = downsampled.drop('target', axis=1)

y_pred_knn = kNN.fit(X_train, y_train).predict(X_test)

print("Malignant in test set : %0.2f" % (1- (y_test.sum()/len(y_test))))
print("Predicted malignant : %0.2f" % (1- (y_pred_knn.sum()/len(y_pred_knn))))

acc = accuracy_score(y_test, y_pred_knn)
print("Downsampling Accuracy hold-out: {:.2f}".format(acc)) 
confusion = confusion_matrix(y_test, y_pred_knn)
print("Downsampling Confusion matrix:\n{}".format(confusion))


#### In this scenario, after upsampling and downsampling we are getting similar results.The accuracy increases from 91% to 92%. However, there is there is a small increase in the True-Positive values. The bias is more towards the majortiy class when compared to that of the imbalanced dataset. In conclusion, there is no much profit from this type of method to balance the dataset as the results could vary depending on the data.




# 3. Test the impact of this strategy on another dataset of your choice. Discuss the effectiveness of the strategy on this second dataset.

#### The dataset considered here is Wine dataset with two classes 0 and 1.

#### In this imbalanced dataset, Class 0 has 43 observations and Class 1 has 61 observations.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score

wineDF = pd.read_csv('wine.csv')
wineDF = wineDF.reset_index(drop=True)
wineDF.head()

y = wineDF['class']
X = wineDF.drop('class', axis=1)

kNN = KNeighborsClassifier(n_neighbors=3)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

print("**** With imbalanced dataset ****")

y_pred_knn = kNN.fit(X_train, y_train).predict(X_test)

print("Malignant in test set : %0.2f" % (1- (y_test.sum()/len(y_test))))
print("Predicted malignant : %0.2f" % (1- (y_pred_knn.sum()/len(y_pred_knn))))

acc = accuracy_score(y_test, y_pred_knn)
print("Accuracy hold-out: {0:.2f}".format(acc)) 

plot_confusion_matrix(y_test, y_pred_knn, classes=['Not Helpful','Helpful'],
                      title='Confusion matrix, without normalization')


# concatenate our training data back together
X = pd.concat([X_train, y_train], axis=1)

# separate minority and majority classes
type1 = X[X['class'] == 0]
type2 = X[X['class'] == 1]


In [ ]:
print('**** Upsampling ****')
# upsample minority
type1_upsampled = resample(type1,
                          replace=True, # sample with replacement
                          n_samples=len(type2), # match number in majority class
                          random_state=27) # reproducible results
# combine majority and upsampled minority
upsampled = pd.concat([type2, type1_upsampled])


y_train = upsampled['class']
X_train = upsampled.drop('class', axis=1)


y_pred_knn = kNN.fit(X_train, y_train).predict(X_test)

print("Malignant in test set : %0.2f" % (1- (y_test.sum()/len(y_test))))
print("Predicted malignant : %0.2f" % (1- (y_pred_knn.sum()/len(y_pred_knn))))

acc = accuracy_score(y_test, y_pred_knn)
print("Upsampling Accuracy hold-out: {:.2f}".format(acc)) 

plot_confusion_matrix(y_test, y_pred_knn, classes=['Not Helpful','Helpful'],
                      title='Confusion matrix, without normalization')


In [ ]:
print('**** Downsampling ****')
#downsample majority
type2_downsampled = resample(type2,
                          replace=True, # sample with replacement
                          n_samples=len(type1), # match number in majority class
                          random_state=27) # reproducible results
# combine majority and upsampled minority
downsampled = pd.concat([type1, type2_downsampled])


y_train = downsampled['class']
X_train = downsampled.drop('class', axis=1)


y_pred_knn = kNN.fit(X_train, y_train).predict(X_test)

print("Malignant in test set : %0.2f" % (1- (y_test.sum()/len(y_test))))
print("Predicted malignant : %0.2f" % (1- (y_pred_knn.sum()/len(y_pred_knn))))

acc = accuracy_score(y_test, y_pred_knn)
print("Upsampling Accuracy hold-out: {:.2f}".format(acc)) 

plot_confusion_matrix(y_test, y_pred_knn, classes=['Not Helpful','Helpful'],
                      title='Confusion matrix, without normalization')


#### The accuracy on testing with imbalanced dataset is 85%.

#### We are handling this imbalance with upsampling and downsampling. The accuracy of the model after upsampling is 92%. Moreover, it is interesting to note that the accuarcy shoots to 100% with downsampling.

#### However, it is not always the case because downsampling leads to removal of useful information which can effect the accuracy.